In [ ]:
def get_stock_and_crypto_data(name,start):
    import yfinance as yf
    from datetime import datetime
    start = datetime.strptime(start, '%Y-%m-%d')# convert string to datetime obj
    # end = datetime.strptime(end, '%Y-%m-%d')# convert string to datetime obj

    data = yf.download(tickers=name, start=start, interval = '60m') #get data
    data = data.loc[data["Volume"] != 0 ].drop(["Adj Close"],axis = 1)# ไม่เอาค่า volume = 0
    data["Date"] = data.index.strftime('%Y-%m-%d %X') # convert pandas timestamp to string 
    return data
    return data.values.tolist() #return value in type list

In [ ]:
get_stock_and_crypto_data("BTC-USD","2021-03-01")

In [ ]:
import yfinance as yf
from datetime import datetime
start = datetime.strptime("2023-01-01", '%Y-%m-%d')# convert string to datetime obj
# end = datetime.strptime(end, '%Y-%m-%d')# convert string to datetime obj

data = yf.Ticker("aapl") #get data


# data = data.loc[data["Volume"] != 0 ].drop(["Adj Close"],axis = 1)# ไม่เอาค่า volume = 0
# data["Date"] = data.index.strftime('%Y-%m-%d %X') # convert pandas timestamp to string
# print(data.values.tolist()) #return value in type list


In [ ]:
def insert_stock(id,data,table):
    from sqlite3 import connect
    conn = connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    for i in data:#run sql script(insert)
        cursor.execute(f"INSERT INTO {table} VALUES (null,{id},'{i[5]}',{i[0]},{i[1]},{i[2]},{i[3]},{i[4]})")
    conn.commit()#commit change to db
    conn.close()#disconnect

In [ ]:
def get_stock_id(symbol):
    from sqlite3 import connect
    conn = connect('stock.db',timeout=10)#connect to database
    cursor = conn.cursor()
    cursor.execute(f"SELECT stock_id FROM stock WHERE symbol = '{symbol}'")#run sql script(get data)
    data = cursor.fetchall()#get data as list
    conn.close()#disconnect
    return data[0][0]

In [ ]:
from data.function import Categories
continues = 0
for i in Categories().get_all_stock():
    if continues == 1:
        insert_stock(get_stock_id(i),get_stock_and_crypto_data(i+'.bk','2021-01-13'),'price_hours')
    elif i == 'THL':
        continues = 1

In [ ]:
import pandas as pd

data = yf.download(tickers="PTT.bk", period="60d", interval = '30m')
data = data.loc[data["Volume"] != 0 ].drop(["Adj Close"],axis = 1)
new_data_open = data['Open'].resample("H").first()

new_data_close = data['Close'].resample("H").last()

new_data_high = data['High'].resample("H").max()
new_data_low = data['Low'].resample("H").min()

new_data_volume = data['Volume'].resample("H").sum()
new_data = pd.DataFrame({'Open': new_data_open,
                                'High': new_data_high,
                                'Low': new_data_low,
                                'Close': new_data_close,
                                'Volume': new_data_volume}).dropna()


new_data["Date"] = new_data.index.strftime('%Y-%m-%d %X') # convert pandas timestamp to string 
print(new_data) #return value in type list